In [16]:
import pandas as pd
import re

In [17]:
reviews_df = pd.read_csv('reviews_shortened.csv')
businesses_df = pd.read_csv('businesses_shortened.csv')

Now we need to classify reviews: using the text, we want to identify the ones in which the reviewer was on a date. 
This is actually a very difficult classification problem, because:

1. All the data is unlabelled. We have no training data.

2. If someone is on a date, it's not clear how apparent that will be in a review. Some reviewers may not allude to that at all. Though including words like "boyfriend" or "girlfriend" would make it clear, we have to assume that a nontrivial portion of the yelp reviews were written by people on dates who did not mention that fact. Conversely, if someone is *not* on a date, it seems quite unlikely for the reviewer to mention that in any explicit way that we can encode in a model. 

While there are other difficulties in binary topic-classification of textual data (e.g. setting thresholds), the two points above confound any sweeping analysis. Though it's possible to use unsupervised/semi-supervised methods to construct some set of words associated with an abstract date-topic, the fundamental issue is that some very significant portion of the reviews may not carry a feature that lets us classify them correctly. In other words, the approach given by "if we detect date-words, classify it as a date, otherwise, classify it as a non-date" may lead to a high number of false negatives.

So we go for a more explicit approach: rather than trying to use the entire dataset, on which any attempt at accurate binary classification will almost certainly fail, we will isolate the reviews that are *certainly* dates and *certainly* non-dates. This will let us make a more explicit comparison, though we may also then use these statistics as reference points when comparing with the corpus of all reviews.

In particular, we identify certain dates as reviews that include the phrase "on a date", which is generally not used in any other context (i.e. the probability of this particular phrase referring to a datetime seems exceedingly low), and the phrase "by myself" to indicate when the reviewer is not on a date. We'll be careful to look out for negation of either of those two phrases.

In [27]:
def by_myself(review_text):
    return review_text.str.contains("by myself") & (~ review_text.str.contains("not by myself"))

non_dates = reviews_df[by_myself(reviews_df['text'])]

def on_a_date(review_text):
    return review_text.str.contains("on a date") & (~ review_text.str.contains("not on a date"))

dates = reviews_df[on_a_date(reviews_df['text'])]

intersection = reviews_df[on_a_date(reviews_df['text']) & by_myself(reviews_df['text'])]

note: it's actually really hard to identify whether someon was on a date. the phrase "on a date" is actually totally useless for this purpose. should use a SO-term, e.g. boyfriend, girlfriend, wife, husband, etc. 

but then we introduce some bias: is a person likely to mention their SO in a review under certain conditions?

This actually gets pretty tricky. needs a more sophisticated linguistic model than I anticipated.

In [28]:
print len(dates)
print len(non_dates)
print len(intersection)

2592
4379
14


In [32]:
list(dates['text'])

['a friend and i dropped by sweetlix on one of those heatindexof120f days because waiting for the aaa guy in the smithfieldliberty garage was about to kill both of us we figured we could cool off with some frozen yogurt i refuse to call it froyo and sit around in the ac for a whilei wasnt sure which flavor i wanted when i went in i asked about the tangy one and the girl described it with some assistance from the other person behind the counter but never offered a sample i ended up with the snickerdoodle which was good but tasted essentially like eggnog ie nutmeg not cinnamon i am in general not a fan of stuff either in or on my frozen dairy products so it was unadorned the texture was great and the taste was amazing too  just not really quite as labeled if i were going to put toppings on this flavor id go with raspberries honey and cinnamon sugar thatd be pretty tastysadly the atmosphere is pretty lacking the ac was barely running so my frozen treat was melting fast while i was standin

In [58]:
us_reviews[:2]

,business_id,date,review_id,stars,text,type,user_id,votes,location
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,Mr Hoagie is an institution Walking in it does...,review,PUFPaY9KxDAcGqfsorJp3Q,"{u'funny': 0, u'useful': 0, u'cool': 0}",USA
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food Superb customer service I miss ...,review,Iu6AxdBYGR4A0wspR9BYHA,"{u'funny': 0, u'useful': 0, u'cool': 0}",USA
